In [ ]:
from keras.models import Sequential, load_model
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy as np
import pandas as pd
import tensorflow as tf

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)

df = pd.read_csv('./dataset/sonar.csv', header=None)

dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

n_fold = 5
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

accuracy = []
index = 0

# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                    optimizer='adam',
                    metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)
    #model.save("./model/lab7-model-" + str(index) + ".hdf5")
    index = index + 1

# 결과 출력
print("\n %.f fold accuracy:" % n_fold, accuracy)
acc = [float (i) for i in accuracy]
maxIdx = acc.index(max(acc))
print("Max index: %d" % maxIdx)

print("Load " + str(maxIdx) + "-th model having " + str(accuracy[maxIdx]) + " accuracy")
print("Sample test of the first 5 element")
del model
model = load_model("./model/lab7-model-" + str(maxIdx) + ".hdf5")
X = X[:5, :60]
Y = Y[:5]
output = model.predict(X)
expect = np.array(output)
for i in range(n_fold):
    print("Index: %d, actual: %.2f, expect: %.2f" % (i, Y[i], expect[i]))